In [1]:
from __future__ import print_function
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import tensorflow as tf
#import keras as kr
import sklearn as skl
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten, BatchNormalization, Conv1D, MaxPooling1D, Activation, UpSampling1D,Reshape
from keras.constraints import maxnorm
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K
from keras import regularizers
from keras.models import load_model
from sklearn import preprocessing
import os
import feather
import keras_metrics
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

Using TensorFlow backend.


In [2]:
#y_validate=pd.read_pickle("CZI.PBMC.y.validate.nn.pkl")
#y_test=pd.read_pickle("CZI.PBMC.y.test.nn.pkl")
#X=pd.read_pickle("CZI.PBMC.full.X.nn.sorted.cleaned.pkl")
#y_train=pd.read_pickle("CZI.PBMC.y.tra")
y_train=pd.read_feather('CZI.PBMC.y.train.nn.feather').values
X_train=pd.read_feather('CZI.PBMC.X.train.nn.feather').values
#X_train=pd.read_feather('CZI.PBMC.X.train.nn.notnorm.feather')


y_validate=pd.read_feather('CZI.PBMC.y.validate.nn.feather').values
X_validate=pd.read_feather('CZI.PBMC.X.validate.nn.feather').values
#X_validate=pd.read_feather('CZI.PBMC.X.validate.nn.notnorm.feather')


N_train_dat=X_train.shape[0]
#N_test_dat=
N_val_dat=X_validate.shape[0]
N_feat=X_validate.shape[1]

#X_train=X_train.values.reshape(N_train_dat,N_feat,1)
#X_validate=X_validate.values.reshape(N_val_dat,N_feat,1)

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_validate_minmax = min_max_scaler.fit_transform(X_validate)

X_train_minmax=X_train_minmax.reshape(N_train_dat,N_feat,1)
X_validate_minmax=X_validate_minmax.reshape(N_val_dat,N_feat,1)


#input_dir = os.getcwd()
#genes=pd.read_csv("genes.nn.csv").x[1:]

#counts_matrix={}
#for i in range(1,11,2):
#    brcds_1=pd.read_csv(input_dir+'/CZI.PBMC.'+str(i)+'.brcds').x
#    counts_matrix_1=scipy.io.mmread(input_dir + '/CZI.PBMC.'+str(i)+'.umis.mtx').T.tocsc()
#    brcds_2=pd.read_csv(input_dir+'/CZI.PBMC.'+str(i+1)+'.brcds').x
#    counts_matrix_2=scipy.io.mmread(input_dir + '/CZI.PBMC.'+str(i+1)+'.umis.mtx').T.tocsc()
#    df_1=pd.DataFrame(counts_matrix_1.todense(),index=brcds_1,columns=genes)
 #   df_2=pd.DataFrame(counts_matrix_2.todense(),index=brcds_2,columns=genes)
#    if i==1:
#        czi_pbmc=pd.concat([df_1,df_2])
#    else:
#        czi_pbmc=pd.concat([czi_pbmc,df_1,df_2])
#        
#        
#czi_pbmc.to_pickle("CZI.PBMC.full.nn.pkl")




#df_x_train = pd.read_csv("/projects/ucar-lab/danaco/bncmrk-dblts/DF/input/CZI.PBMC/CZI.PBMC.2.train.csv", header=None)
#df_x_validate = pd.read_csv("/projects/ucar-lab/danaco/bncmrk-dblts/DF/input/CZI.PBMC/CZI.PBMC.2.validate.csv", header=None)
#df_x_test = pd.read_csv("/projects/ucar-lab/danaco/bncmrk-dblts/DF/input/CZI.PBMC/CZI.PBMC.2.test.csv", header=None)



In [3]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(" — vali_f1: %f — vali_precision: %f — vali_recall %f"%(_val_f1, _val_precision, _val_recall))
        return
 

metricS = Metrics()

In [4]:
#train_locs=np.array([np.where(~all_brcds.Barcode.isin(val_tst_brcds.Barcode).values)])[0,0,:]
#all_brcds.shape
#not True
#train_brcds=all_brcds.iloc[train_locs,]
#pd.DataFrame(train_brcds.Barcode.values).to_pickle("CZI.PBMC.train.barcodes.pkl")
os.listdir()

['dnn_autoencoder_clr.h5',
 'CZI.PBMC.X.validate.nn2.pkl',
 'CZI.PBMC.full.Y.nn.sorted.cleaned.csv',
 'CZI.PBMC.new.validate.barcodes.pkl',
 'cnn_autoencoder_plain_moreepoch.h5',
 'iris.csv',
 'CZI.PBMC.5.brcds',
 '.ipynb_checkpoints',
 'CZI.PBMC.8.brcds',
 'CZI.PBMC.test.barcodes.pkl',
 'CZI.PBMC.2.umis.mtx',
 'CZI.PBMC.3.brcds',
 'CZI.PBMC.3.test-Copy1.csv',
 'CZI.PBMC.X.validate.nn.csv',
 'CZI.PBMC.9.umis.mtx',
 'CZI.PBMC.X.test.nn2.pkl',
 'data_line_save_feather.o10596475',
 'data_line_save3.py',
 'CZI.PBMC.full.X.nn.pkl',
 'CZI.PBMC.X.validate.nn.notnorm.feather',
 'cnn_autoencoder_lasso.h5',
 'CZI.PBMC.X.test.nn.feather',
 'CZI.PBMC.full.X.nn.sorted.cleaned.pkl',
 'CZI.PBMC.X.validate.nn.feather',
 'CZI.PBMC.y.train.nn.feather',
 'CZI.PBMC.train.barcodes.pkl',
 'CZI.PBMC.new.test.barcodes.pkl',
 'czi_pbmc_notnorm.mtx',
 'data_prep_pbs.sh',
 'first_try.ipynb',
 'data_line_save_pbs2.sh',
 'CZI.PBMC.X.test.nn.pkl',
 'sonar_tutorial.ipynb',
 'CZI.PBMC.full.X.nn.sorted.cleaned.feather

In [4]:
#bebe=pd.read_pickle("CZI.PBMC.new.train.locs.pkl")

#pd.DataFrame(train_locs,columns=np.array(["Locs"])).to_pickle("CZI.PBMC.new.train.locs.pkl")
#pd.DataFrame(y.index.values[train_locs],columns=np.array(["Barcode"])).to_pickle("CZI.PBMC.new.train.barcodes.pkl")

#pd.DataFrame(validate_locs,columns=np.array(["Locs"])).to_pickle("CZI.PBMC.new.validate.locs.pkl")
#pd.DataFrame(y.index.values[validate_locs],columns=np.array(["Barcode"])).to_pickle("CZI.PBMC.new.validate.barcodes.pkl")

#pd.DataFrame(test_locs,columns=np.array(["Locs"])).to_pickle("CZI.PBMC.new.test.locs.pkl")
#pd.DataFrame(y.index.values[test_locs],columns=np.array(["Barcode"])).to_pickle("CZI.PBMC.new.test.barcodes.pkl")
cnn_autoencoder=load_model('cnn_autoencoder_plain_moreepoch.h5')

W0829 06:51:20.117908 46912496324416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 06:51:20.159262 46912496324416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 06:51:20.403329 46912496324416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 06:51:20.404419 46912496324416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0829 06:51:20.421202 469

In [5]:
from keras.callbacks import TensorBoard, EarlyStopping
from tensorboard.plugins.pr_curve import summary as pr_summary

# Check complete example in:
# https://github.com/akionakamura/pr-tensorboard-keras-example
class PRTensorBoard(TensorBoard):
    def __init__(self, *args, **kwargs):
        # One extra argument to indicate whether or not to use the PR curve summary.
        self.pr_curve = kwargs.pop('pr_curve', True)
        super(PRTensorBoard, self).__init__(*args, **kwargs)

        global tf
        import tensorflow as tf

    def set_model(self, model):
        super(PRTensorBoard, self).set_model(model)

        if self.pr_curve:
            # Get the prediction and label tensor placeholders.
            predictions = self.model._feed_outputs[0]
            labels = tf.cast(self.model._feed_targets[0], tf.bool)
            # Create the PR summary OP.
            self.pr_summary = pr_summary.op(tag='pr_curve',
                                            predictions=predictions,
                                            labels=labels,
                                            display_name='Precision-Recall Curve')

    def on_epoch_end(self, epoch, logs=None):
        super(PRTensorBoard, self).on_epoch_end(epoch, logs)

        if self.pr_curve and self.validation_data:
            # Get the tensors again.
            tensors = self.model._feed_targets + self.model._feed_outputs
            # Predict the output.
            predictions = self.model.predict(self.validation_data[:-2])
            # Build the dictionary mapping the tensor to the data.
            val_data = [self.validation_data[-2], predictions]
            feed_dict = dict(zip(tensors, val_data))
            # Run and add summary.
            result = self.sess.run([self.pr_summary], feed_dict=feed_dict)
            self.writer.add_summary(result[0], epoch)
            self.writer.flush()

In [9]:
# The known number of output classes.
#num_classes = 2

# Input image dimensions
#input_shape = (18488,)

##cnn_autoencoder=load_model('cnn_autoencoder_plain_moreepoch.h5')

#cnn_autoencoder = Sequential()

#cnn_autoencoder.add(Conv1D(128,kernel_size=3, input_shape=(N_feat,1), use_bias=False,padding='same',kernel_initializer="uniform"))

#cnn_autoencoder.add(BatchNormalization())
#cnn_autoencoder.add(Activation('relu'))
#cnn_autoencoder.add(ActivityRegularization(l1=10e-6,l2=10e-6))
#cnn_autoencoder.add(MaxPooling1D(pool_size=2))

#cnn_autoencoder.add(Conv1D(64,kernel_size=3,kernel_initializer='uniform',activation='relu',padding='same'))
#cnn_autoencoder.add(Dropout(0.2))
#cnn_autoencoder.add(MaxPooling1D(pool_size=2))

#cnn_autoencoder.add(Conv1D(32,kernel_size=3,kernel_initializer='uniform',activation='relu',padding='same'))

#cnn_autoencoder.add(UpSampling1D(2))

#cnn_autoencoder.add(Conv1D(64,kernel_size=3,kernel_initializer='uniform',activation='relu',padding='same'))

#cnn_autoencoder.add(UpSampling1D(2))
#cnn_autoencoder.add(Conv1D(1,kernel_size=3,kernel_initializer='uniform',activation='sigmoid',padding="same"))




#cnn_autoencoder.compile(loss=keras.losses.binary_crossentropy, #mean_squared_error
 #             optimizer=keras.optimizers.Adam(),
 #             metrics=['accuracy'])

###cnn_autoencoder.summary()

encoder_layer0 = cnn_autoencoder.layers[0]
encoder_layer0.trainable=False
encoder_layer1 = cnn_autoencoder.layers[1]
encoder_layer1.trainable=False
encoder_layer2 = cnn_autoencoder.layers[2]
encoder_layer2.trainable=False
encoder_layer3 = cnn_autoencoder.layers[3]
encoder_layer3.trainable=False
encoder_layer4 = cnn_autoencoder.layers[4]
encoder_layer4.trainable=False
encoder_layer5 = cnn_autoencoder.layers[5]
encoder_layer5.trainable=False
encoder_layer6 = cnn_autoencoder.layers[6]
encoder_layer6.trainable=False
#dnn_encoder = Model(input_img, encoder_layer5(encoder_layer4(encoder_layer3(encoder_layer2(encoder_layer1(input_img))))))


class_model=Sequential()
class_model.add(encoder_layer0)
class_model.add(encoder_layer1)
class_model.add(encoder_layer2)
class_model.add(encoder_layer3)
class_model.add(encoder_layer4)
class_model.add(encoder_layer5)
class_model.add(encoder_layer6)
class_model.add(MaxPooling1D(2))
class_model.add(Conv1D(1,3))
#class_model.add(MaxPooling1D(2))
class_model.add(Flatten())
#class_model.add(Dense(1024,activation='relu'))
#class_model.add(Dropout(0.2))
class_model.add(Dense(512,activation='relu'))#,activity_regularizer=regularizers.l1(20e-6)))
#class_model.add(Dropout(0.2))
class_model.add(Dense(512,activation='relu'))
class_model.add(Dense(1,activation='sigmoid'))


class_model.compile(loss=keras.losses.binary_crossentropy, #mean_squared_error
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy',keras_metrics.binary_precision(), keras_metrics.binary_recall(),keras_metrics.binary_f1_score()])#,
                  # callbacks=[metricS])

class_model.summary()






#encoder.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_63 (Conv1D)           (None, 18488, 128)        384       
_________________________________________________________________
batch_normalization_19 (Batc (None, 18488, 128)        512       
_________________________________________________________________
activation_19 (Activation)   (None, 18488, 128)        0         
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 9244, 128)         0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 9244, 64)          24640     
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 4622, 64)          0         
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 4622, 32)          6176      
__________

In [ ]:
current_dir = os.getcwd()
log_dir = os.path.join(current_dir, 'logs')

if not os.path.exists(log_dir) or not os.path.isdir(log_dir):
    os.mkdir(log_dir)

N0=np.sum(y_validate==0)
N1=np.sum(y_validate==1)
w0 = (1 / N0) / (0.5 * (1 / N0 + 1 / N1))
w1 = (1 / N1) / (0.5 * (1 / N0 + 1 / N1))
class_weights={0:w0,1:w1}

batch_size = 256
epochs = 50

#callback = [ PRTensorBoard(log_dir=log_dir), EarlyStopping(monitor='val_loss', patience=3) ]

callback = None#[ EarlyStopping(monitor='val_loss', patience=4) ]


history=class_model.fit(X_train_minmax, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validate_minmax, y_validate),class_weight=class_weights,
                       callbacks=callback)

allDone()

Train on 242744 samples, validate on 30343 samples
Epoch 1/50
242744/242744 [==============================] - 518s 2ms/step - loss: 0.5306 - acc: 0.6628 - precision: 0.4135 - recall: 0.5533 - f1_score: 0.4733 - val_loss: 0.5979 - val_acc: 0.7106 - val_precision: 0.4765 - val_recall: 0.4785 - val_f1_score: 0.4775
Epoch 2/50
242744/242744 [==============================] - 515s 2ms/step - loss: 0.4995 - acc: 0.6916 - precision: 0.4471 - recall: 0.5355 - f1_score: 0.4874 - val_loss: 0.6326 - val_acc: 0.6786 - val_precision: 0.4359 - val_recall: 0.5536 - val_f1_score: 0.4877
Epoch 3/50
242744/242744 [==============================] - 521s 2ms/step - loss: 0.4923 - acc: 0.6981 - precision: 0.4569 - recall: 0.5444 - f1_score: 0.4969 - val_loss: 0.6363 - val_acc: 0.6590 - val_precision: 0.4175 - val_recall: 0.5921 - val_f1_score: 0.4897
Epoch 4/50
242744/242744 [==============================] - 517s 2ms/step - loss: 0.4848 - acc: 0.7013 - precision: 0.4629 - recall: 0.5680 - f1_score: 0.510

In [ ]:

df1_train=pd.DataFrame(X_train_minmax)
df1_validate=pd.DataFrame(X_validate_minmax)


In [ ]:
#X_validate_minmax_pred=np.array(dnn_autoencoder_notnorm.predict(X_validate_minmax))
X_validate_minmax_pred=np.array(cnn_autoencoder.predict(X_validate_minmax,batch_size=256))
X_train_minmax_pred=np.array(cnn_autoencoder.predict(X_train_minmax,batch_size=256))

In [ ]:
pd.DataFrame(X_validate_minmax[10,]).hist()

In [ ]:
X_train_minmax_pred=np.array(X_train_minmax_pred)

In [ ]:
pd.DataFrame(X_validate_minmax_pred[10,]).hist()

In [ ]:
#X=scipy.io.mmread('czi_pbmc_notnorm.mtx').todense()
X=scipy.io.mmread('CZI.PBMC.1.umis.mtx').todense()

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
X_train=X_train.reshape(N_train_dat,N_feat,1)
X_validate=X_validate.reshape(N_val_dat,N_feat,1)

In [ ]:
from keras.layers import ActivityRegularization

In [ ]:
cnn_autoencoder.save('cnn_autoencoder_plain_moreepoch.h5')

In [ ]:
encoder_layer0.layer_trainable

In [ ]:
X_validate_encode_pred=np.array(class_model.predict(X_validate_minmax,batch_size=256))

In [ ]:
del encoder_layer0, encoder_layer1, encoder_layer2, encoder_layer3, encoder_layer4, encoder_layer5, encoder_layer6

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [ ]:
from sklearn.utils import class_weight


In [ ]:
np.sum(y_validate==0)/np.sum(y_validate==1)

In [ ]:
N0=np.sum(y_validate==0)
N1=np.sum(y_validate==1)
w0 = (1 / N0) / (0.5 * (1 / N0 + 1 / N1))
w1 = (1 / N1) / (0.5 * (1 / N0 + 1 / N1))
class_weights={0:w0,1:w1}

In [19]:
class_model.layers[7].trainable

True

In [30]:
y_train.shape

(242744, 1)

In [4]:
    keras_metrics.bin

<function keras_metrics.metric_fn.<locals>.fn(label=0, **kwargs)>

In [40]:
current_dir = os.getcwd()
log_dir = os.path.join(current_dir, 'logs')

if not os.path.exists(log_dir) or not os.path.isdir(log_dir):
    os.mkdir(log_dir)

In [38]:
os.listdir()

['dnn_autoencoder_clr.h5',
 'CZI.PBMC.X.validate.nn2.pkl',
 'CZI.PBMC.full.Y.nn.sorted.cleaned.csv',
 'CZI.PBMC.new.validate.barcodes.pkl',
 'cnn_autoencoder_plain_moreepoch.h5',
 'iris.csv',
 'CZI.PBMC.5.brcds',
 '.ipynb_checkpoints',
 'CZI.PBMC.8.brcds',
 'CZI.PBMC.test.barcodes.pkl',
 'CZI.PBMC.2.umis.mtx',
 'CZI.PBMC.3.brcds',
 'CZI.PBMC.3.test-Copy1.csv',
 'CZI.PBMC.X.validate.nn.csv',
 'CZI.PBMC.9.umis.mtx',
 'CZI.PBMC.X.test.nn2.pkl',
 'data_line_save_feather.o10596475',
 'data_line_save3.py',
 'CZI.PBMC.full.X.nn.pkl',
 'CZI.PBMC.X.validate.nn.notnorm.feather',
 'cnn_autoencoder_lasso.h5',
 'CZI.PBMC.X.test.nn.feather',
 'CZI.PBMC.full.X.nn.sorted.cleaned.pkl',
 'CZI.PBMC.X.validate.nn.feather',
 'CZI.PBMC.y.train.nn.feather',
 'CZI.PBMC.train.barcodes.pkl',
 'CZI.PBMC.new.test.barcodes.pkl',
 'czi_pbmc_notnorm.mtx',
 'data_prep_pbs.sh',
 'first_try.ipynb',
 'data_line_save_pbs2.sh',
 'CZI.PBMC.X.test.nn.pkl',
 'sonar_tutorial.ipynb',
 'CZI.PBMC.full.X.nn.sorted.cleaned.feather

In [12]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above